In [1]:
import pandas as pd
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

In [2]:
# Standard
import pandas as pd
import numpy as np
import os
# Pycaret
from pycaret.classification import *
# Plots
from plotly.offline import iplot
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
# Sklearn tools
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import *
# Extras
from datetime import date
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import pycaret

from sklearn.impute import SimpleImputer

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [3]:
df_test = pd.read_csv('test.csv')

In [4]:
df_train = pd.read_csv('train.csv')

In [5]:
df_train

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,...,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
0,0,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,31.277,87.009,...,0,0,NaN,NaN,NaN,1958,0.00,0.00,0.00,0
1,1,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,0.000,122.787,...,0,0,NaN,1.0,NaN,710,0.00,0.00,0.00,0
2,2,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,60.806,103.176,...,0,0,NaN,NaN,NaN,882,0.00,0.00,0.00,0
3,3,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,156.362,205.260,...,0,0,NaN,NaN,NaN,982,0.00,0.00,0.00,0
4,4,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,240.708,128.191,...,1,0,1.0,1.0,1.0,647,0.00,0.00,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69994,69994,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,15.760,410.924,...,1,0,NaN,1.0,1.0,221,0.00,0.00,0.00,0
69995,69995,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,160.083,289.129,...,0,0,NaN,NaN,NaN,712,0.00,0.00,0.00,0
69996,69996,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,372.088,258.374,...,0,0,NaN,NaN,NaN,879,0.00,0.00,0.00,0
69997,69997,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,238.575,245.414,...,0,0,1.0,1.0,1.0,277,664.25,1402.96,990.97,0


In [6]:
percent_missing = df_train.isnull().sum() * 100 / len(df_train)
missing_value_df = pd.DataFrame({'column_name': df_train.columns,
                                 'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
id,id,0.000000
circle_id,circle_id,0.000000
loc_og_t2o_mou,loc_og_t2o_mou,1.002871
std_og_t2o_mou,std_og_t2o_mou,1.002871
loc_ic_t2o_mou,loc_ic_t2o_mou,1.002871
...,...,...
aon,aon,0.000000
aug_vbc_3g,aug_vbc_3g,0.000000
jul_vbc_3g,jul_vbc_3g,0.000000
jun_vbc_3g,jun_vbc_3g,0.000000


In [19]:
columns_with_good_data = missing_value_df[missing_value_df['percent_missing']<40]['column_name'].to_list()


In [21]:
df_train_short = df_train[columns_with_good_data]

In [22]:
columns_with_good_data.remove('churn_probability')

In [23]:
df_test_short = df_test[columns_with_good_data]

In [24]:
df_test_short['churn_probability'] = -1

In [25]:
df_short = pd.concat([df_train_short,df_test_short])

In [27]:
df_short.drop(columns=['last_date_of_month_6'],inplace=True)

In [28]:
df_short.loc[df_short['last_date_of_month_7'].notnull(), 'last_date_of_month_7'] = 1
df_short.loc[df_short['last_date_of_month_7'].isnull(), 'last_date_of_month_7'] = 0

In [29]:
df_short.loc[df_short['last_date_of_month_8'].notnull(), 'last_date_of_month_8'] = 1
df_short.loc[df_short['last_date_of_month_8'].isnull(), 'last_date_of_month_8'] = 0

In [30]:
df_short

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,arpu_8,...,monthly_3g_7,monthly_3g_8,sachet_3g_6,sachet_3g_7,sachet_3g_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
0,0,109,0.0,0.0,0.0,1,1,31.277,87.009,7.527,...,0,0,0,0,0,1958,0.00,0.00,0.00,0
1,1,109,0.0,0.0,0.0,1,1,0.000,122.787,42.953,...,1,0,0,0,0,710,0.00,0.00,0.00,0
2,2,109,0.0,0.0,0.0,1,1,60.806,103.176,0.000,...,0,0,0,0,0,882,0.00,0.00,0.00,0
3,3,109,0.0,0.0,0.0,1,1,156.362,205.260,111.095,...,0,0,0,0,0,982,0.00,0.00,0.00,0
4,4,109,0.0,0.0,0.0,1,1,240.708,128.191,101.565,...,0,0,0,1,0,647,0.00,0.00,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,99994,109,0.0,0.0,0.0,1,1,718.870,396.259,406.150,...,0,0,0,0,0,1255,3.71,105.64,61.48,-1
29996,99995,109,0.0,0.0,0.0,1,1,218.327,324.070,374.981,...,1,0,0,0,0,2258,0.00,0.00,0.00,-1
29997,99996,109,0.0,0.0,0.0,1,1,139.473,38.230,180.194,...,0,0,0,0,0,440,0.00,0.00,0.00,-1
29998,99997,109,0.0,0.0,0.0,1,1,1122.912,781.121,257.439,...,0,0,0,0,0,810,0.00,0.00,0.00,-1


In [ ]:
#Fill NaN Values

In [33]:
percent_missing = df_short.isnull().sum() * 100 / len(df_short)
missing_value_df = pd.DataFrame({'column_name': df_short.columns,
                                 'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
id,id,0.00000
circle_id,circle_id,0.00000
loc_og_t2o_mou,loc_og_t2o_mou,1.01801
std_og_t2o_mou,std_og_t2o_mou,1.01801
loc_ic_t2o_mou,loc_ic_t2o_mou,1.01801
...,...,...
aon,aon,0.00000
aug_vbc_3g,aug_vbc_3g,0.00000
jul_vbc_3g,jul_vbc_3g,0.00000
jun_vbc_3g,jun_vbc_3g,0.00000


In [34]:
df_short.drop(columns = ['date_of_last_rech_6','date_of_last_rech_7','date_of_last_rech_8'], inplace=True)

In [31]:
imp = IterativeImputer(max_iter=10, random_state=0)

In [35]:
imp.fit(df_short)

IterativeImputer(random_state=0)

In [36]:
df_short_clean = imp.fit_transform(df_short)

In [38]:
df_short_clean

array([[ 0.0000e+00,  1.0900e+02,  0.0000e+00, ...,  0.0000e+00,
         0.0000e+00,  0.0000e+00],
       [ 1.0000e+00,  1.0900e+02,  0.0000e+00, ...,  0.0000e+00,
         0.0000e+00,  0.0000e+00],
       [ 2.0000e+00,  1.0900e+02,  0.0000e+00, ...,  0.0000e+00,
         0.0000e+00,  0.0000e+00],
       ...,
       [ 9.9996e+04,  1.0900e+02,  0.0000e+00, ...,  0.0000e+00,
         0.0000e+00, -1.0000e+00],
       [ 9.9997e+04,  1.0900e+02,  0.0000e+00, ...,  0.0000e+00,
         0.0000e+00, -1.0000e+00],
       [ 9.9998e+04,  1.0900e+02,  0.0000e+00, ...,  6.8713e+02,
         2.2760e+01, -1.0000e+00]])

In [49]:
df_short.columns.to_list()

['id',
 'circle_id',
 'loc_og_t2o_mou',
 'std_og_t2o_mou',
 'loc_ic_t2o_mou',
 'last_date_of_month_7',
 'last_date_of_month_8',
 'arpu_6',
 'arpu_7',
 'arpu_8',
 'onnet_mou_6',
 'onnet_mou_7',
 'onnet_mou_8',
 'offnet_mou_6',
 'offnet_mou_7',
 'offnet_mou_8',
 'roam_ic_mou_6',
 'roam_ic_mou_7',
 'roam_ic_mou_8',
 'roam_og_mou_6',
 'roam_og_mou_7',
 'roam_og_mou_8',
 'loc_og_t2t_mou_6',
 'loc_og_t2t_mou_7',
 'loc_og_t2t_mou_8',
 'loc_og_t2m_mou_6',
 'loc_og_t2m_mou_7',
 'loc_og_t2m_mou_8',
 'loc_og_t2f_mou_6',
 'loc_og_t2f_mou_7',
 'loc_og_t2f_mou_8',
 'loc_og_t2c_mou_6',
 'loc_og_t2c_mou_7',
 'loc_og_t2c_mou_8',
 'loc_og_mou_6',
 'loc_og_mou_7',
 'loc_og_mou_8',
 'std_og_t2t_mou_6',
 'std_og_t2t_mou_7',
 'std_og_t2t_mou_8',
 'std_og_t2m_mou_6',
 'std_og_t2m_mou_7',
 'std_og_t2m_mou_8',
 'std_og_t2f_mou_6',
 'std_og_t2f_mou_7',
 'std_og_t2f_mou_8',
 'std_og_t2c_mou_6',
 'std_og_t2c_mou_7',
 'std_og_t2c_mou_8',
 'std_og_mou_6',
 'std_og_mou_7',
 'std_og_mou_8',
 'isd_og_mou_6',
 'isd_og_

In [50]:
df_short_clean_df = pd.DataFrame(df_short_clean , columns=df_short.columns.to_list())

In [51]:
df_short_clean_df

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,arpu_8,...,monthly_3g_7,monthly_3g_8,sachet_3g_6,sachet_3g_7,sachet_3g_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
0,0.0,109.0,0.0,0.0,0.0,1.0,1.0,31.277,87.009,7.527,...,0.0,0.0,0.0,0.0,0.0,1958.0,0.00,0.00,0.00,0.0
1,1.0,109.0,0.0,0.0,0.0,1.0,1.0,0.000,122.787,42.953,...,1.0,0.0,0.0,0.0,0.0,710.0,0.00,0.00,0.00,0.0
2,2.0,109.0,0.0,0.0,0.0,1.0,1.0,60.806,103.176,0.000,...,0.0,0.0,0.0,0.0,0.0,882.0,0.00,0.00,0.00,0.0
3,3.0,109.0,0.0,0.0,0.0,1.0,1.0,156.362,205.260,111.095,...,0.0,0.0,0.0,0.0,0.0,982.0,0.00,0.00,0.00,0.0
4,4.0,109.0,0.0,0.0,0.0,1.0,1.0,240.708,128.191,101.565,...,0.0,0.0,0.0,1.0,0.0,647.0,0.00,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,99994.0,109.0,0.0,0.0,0.0,1.0,1.0,718.870,396.259,406.150,...,0.0,0.0,0.0,0.0,0.0,1255.0,3.71,105.64,61.48,-1.0
99995,99995.0,109.0,0.0,0.0,0.0,1.0,1.0,218.327,324.070,374.981,...,1.0,0.0,0.0,0.0,0.0,2258.0,0.00,0.00,0.00,-1.0
99996,99996.0,109.0,0.0,0.0,0.0,1.0,1.0,139.473,38.230,180.194,...,0.0,0.0,0.0,0.0,0.0,440.0,0.00,0.00,0.00,-1.0
99997,99997.0,109.0,0.0,0.0,0.0,1.0,1.0,1122.912,781.121,257.439,...,0.0,0.0,0.0,0.0,0.0,810.0,0.00,0.00,0.00,-1.0


In [45]:
df_short_clean_df.dtypes

0      float64
1      float64
2      float64
3      float64
4      float64
        ...   
133    float64
134    float64
135    float64
136    float64
137    float64
Length: 138, dtype: object

In [52]:
df_short_clean_df.to_csv( 'df_short_clean_df.csv', index=False)

In [53]:
df_short_clean_train =  df_short_clean_df[df_short_clean_df['churn_probability'] != 0]

In [58]:
df_short_clean_test =  df_short_clean_df[df_short_clean_df['churn_probability'] == 0]

In [59]:
df_short_clean_test

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,arpu_8,...,monthly_3g_7,monthly_3g_8,sachet_3g_6,sachet_3g_7,sachet_3g_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
0,0.0,109.0,0.0,0.0,0.0,1.0,1.0,31.277,87.009,7.527,...,0.0,0.0,0.0,0.0,0.0,1958.0,0.00,0.00,0.00,0.0
1,1.0,109.0,0.0,0.0,0.0,1.0,1.0,0.000,122.787,42.953,...,1.0,0.0,0.0,0.0,0.0,710.0,0.00,0.00,0.00,0.0
2,2.0,109.0,0.0,0.0,0.0,1.0,1.0,60.806,103.176,0.000,...,0.0,0.0,0.0,0.0,0.0,882.0,0.00,0.00,0.00,0.0
3,3.0,109.0,0.0,0.0,0.0,1.0,1.0,156.362,205.260,111.095,...,0.0,0.0,0.0,0.0,0.0,982.0,0.00,0.00,0.00,0.0
4,4.0,109.0,0.0,0.0,0.0,1.0,1.0,240.708,128.191,101.565,...,0.0,0.0,0.0,1.0,0.0,647.0,0.00,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69994,69994.0,109.0,0.0,0.0,0.0,1.0,1.0,15.760,410.924,329.136,...,0.0,0.0,0.0,1.0,0.0,221.0,0.00,0.00,0.00,0.0
69995,69995.0,109.0,0.0,0.0,0.0,1.0,1.0,160.083,289.129,265.772,...,0.0,0.0,0.0,0.0,0.0,712.0,0.00,0.00,0.00,0.0
69996,69996.0,109.0,0.0,0.0,0.0,1.0,1.0,372.088,258.374,279.782,...,0.0,0.0,0.0,0.0,0.0,879.0,0.00,0.00,0.00,0.0
69997,69997.0,109.0,0.0,0.0,0.0,1.0,1.0,238.575,245.414,145.062,...,1.0,1.0,0.0,0.0,0.0,277.0,664.25,1402.96,990.97,0.0


In [62]:
df_short[df_short['churn_probability'] != 0]

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,arpu_8,...,monthly_3g_7,monthly_3g_8,sachet_3g_6,sachet_3g_7,sachet_3g_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
8,8,109,0.0,0.0,0.0,1,1,163.550,90.250,51.726,...,0,0,0,0,0,2455,0.00,0.00,0.00,1
31,31,109,0.0,0.0,0.0,1,1,894.407,126.500,255.700,...,0,0,0,0,0,303,0.00,0.00,0.00,1
33,33,109,0.0,0.0,0.0,1,1,438.077,0.000,213.781,...,0,1,0,0,0,457,0.00,0.00,0.00,1
41,41,109,0.0,0.0,0.0,1,1,53.997,131.395,0.000,...,0,0,0,0,0,194,0.00,0.00,0.00,1
55,55,109,0.0,0.0,0.0,1,1,86.986,120.089,105.035,...,0,0,0,0,0,789,0.00,0.00,0.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,99994,109,0.0,0.0,0.0,1,1,718.870,396.259,406.150,...,0,0,0,0,0,1255,3.71,105.64,61.48,-1
29996,99995,109,0.0,0.0,0.0,1,1,218.327,324.070,374.981,...,1,0,0,0,0,2258,0.00,0.00,0.00,-1
29997,99996,109,0.0,0.0,0.0,1,1,139.473,38.230,180.194,...,0,0,0,0,0,440,0.00,0.00,0.00,-1
29998,99997,109,0.0,0.0,0.0,1,1,1122.912,781.121,257.439,...,0,0,0,0,0,810,0.00,0.00,0.00,-1


In [64]:
df_test

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,...,sachet_3g_6,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g
0,69999,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,91.882,65.330,...,0,0,0,NaN,NaN,NaN,1692,0.00,0.00,0.00
1,70000,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,414.168,515.568,...,0,0,0,NaN,NaN,NaN,2533,0.00,0.00,0.00
2,70001,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,329.844,434.884,...,0,0,0,NaN,NaN,NaN,277,525.61,758.41,241.84
3,70002,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,43.550,171.390,...,0,0,0,NaN,NaN,NaN,1244,0.00,0.00,0.00
4,70003,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,306.854,406.289,...,0,0,0,NaN,NaN,NaN,462,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,99994,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,718.870,396.259,...,0,0,0,1.0,NaN,NaN,1255,3.71,105.64,61.48
29996,99995,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,218.327,324.070,...,0,0,0,NaN,0.0,NaN,2258,0.00,0.00,0.00
29997,99996,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,139.473,38.230,...,0,0,0,NaN,NaN,NaN,440,0.00,0.00,0.00
29998,99997,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,1122.912,781.121,...,0,0,0,NaN,NaN,NaN,810,0.00,0.00,0.00


In [57]:
df_short_clean_df['churn_probability'].unique()

array([ 0.,  1., -1.])

In [ ]:
df_short_clean_df.to_csv( 'df_short_clean_df.csv', index=False)

In [42]:
percent_missing = df_short_clean_df.isnull().sum() * 100 / len(df_short_clean_df)
missing_value_df = pd.DataFrame({'column_name': df_short_clean_df.columns,
                                 'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
133,133,0.0
134,134,0.0
135,135,0.0
136,136,0.0


In [ ]:
df_test_short = df_test[columns_with_good_data]

In [ ]:
df_short['last_date_of_month_9'].value_counts()

In [ ]:
df_test_short

In [ ]:
df_test_short[df_test_short['last_date_of_month_7'].notnull()].last_date_of_month_7 =1